In [59]:
# Imports
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

# Creating/Cleaning Text DF

In [60]:
from sec_api import QueryApi
queryApi = QueryApi(api_key="43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")

In [61]:
ticker_list = pd.read_csv("./Data/Cleaned_Data/Ticker_library.csv")["Ticker"].to_list()

In [62]:
def SEC_query(ticker_list,filingType):
    file_dict = {}
    for ticker in ticker_list:
        query = {
            "query": { "query_string": { 
                "query": f"ticker:{ticker} AND filedAt:"+"{2012-06-01 TO 2022-06-01} AND formType:\""+f"{filingType}\"" 
            } },
            "from": "0",
            "size": "60",
            "sort": [{ "filedAt": { "order": "desc" } }]
        }
        filings = queryApi.get_filings(query)
        file_dict[ticker] = pd.json_normalize(filings, record_path =['filings'])
    return file_dict

In [63]:
file_10q_df = SEC_query(ticker_list,'10-Q')
file_10k_df = SEC_query(ticker_list,'10-K')

In [64]:
url_data = {
    "ticker":[],
    "periodOfReport":[],
    "linkToFilingDetails":[],
    "formType":[],
    "cik":[]
    }
def url_clean(file_10q_df,url_data):
    for ticker in ticker_list:
        url_data["ticker"].extend(file_10q_df[ticker]["ticker"].to_list())
        url_data["periodOfReport"].extend(file_10q_df[ticker]["periodOfReport"].to_list())
        url_data["linkToFilingDetails"].extend(file_10q_df[ticker]["linkToFilingDetails"].to_list())
        url_data["formType"].extend(file_10q_df[ticker]["formType"].to_list())
        url_data["cik"].extend(file_10q_df[ticker]["cik"].to_list())
url_clean(file_10q_df,url_data)
url_clean(file_10k_df,url_data)

url_df = pd.DataFrame(url_data)

In [65]:
from sec_api import ExtractorApi
sec_api = os.getenv("sec_api_key")
extractorApi = ExtractorApi("43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")

In [66]:
def sec_text_pull(url,filingtype):
    url = str(url)
    data = {}
    if (filingtype == "10-Q"):
        data['Business']=pd.NA
        data['Risk Factors']=extractorApi.get_section(url, "part2item1A", "text")
        data['Management’s Discussion and Analysis of Financial Condition and Results of Operations']=extractorApi.get_section(url, "part1item2", "text")
    else:
        data['Business']=extractorApi.get_section(url, "1", "text")
        data['Risk Factors']=extractorApi.get_section(url, "1A", "text")
        data['Management’s Discussion and Analysis of Financial Condition and Results of Operations']=extractorApi.get_section(url, "7", "text")
    return data

In [67]:
def pull_sec_from_df(df:pd.DataFrame):
    data = {
        "Business":[],
        "Risk Factors":[],
        'Management’s Discussion and Analysis of Financial Condition and Results of Operations':[]
    }

    for rowNumber in range(len(df.index)):
        pull = sec_text_pull(df['linkToFilingDetails'].loc[rowNumber],df['formType'].loc[rowNumber])
        data['Business'].append(pull['Business'])
        data['Risk Factors'].append(pull['Risk Factors'])
        data['Management’s Discussion and Analysis of Financial Condition and Results of Operations'].append(
            pull['Management’s Discussion and Analysis of Financial Condition and Results of Operations'])
    return pd.DataFrame(data)

# Further Data Cleaning and Saving

In [68]:
# sec_text = pull_sec_from_df(url_df)
# sec_text.to_csv('.\Data\Cleaned_Data\sec_text_pulls.csv')
sec_text = pd.read_csv('.\Data\Cleaned_Data\sec_text_pulls.csv',index_col=0)

In [69]:
sec_text_df = pd.concat([url_df,sec_text],axis=1) # Contains undefined text sections
# If Buisness is undefined. Filing does not contain ANY text that is useful, remove
sec_text_df=sec_text_df[sec_text_df['Business']!='undefined']

In [70]:
def create_sec_text_df(ticker_list):
    file_10q_df = SEC_query(ticker_list,'10-Q')
    file_10k_df = SEC_query(ticker_list,'10-K')
    url_data = {
        "ticker":[],
        "periodOfReport":[],
        "linkToFilingDetails":[],
        "formType":[],
        "cik":[]
    }
    url_clean(file_10q_df,url_data)
    url_clean(file_10k_df,url_data)

    url_df = pd.DataFrame(url_data)

    return pd.concat([url_df,pull_sec_from_df(url_df)])

In [71]:
# A quick way for others to grab the library for sentiment analysis
import libs.sec as sec
sec_model = sec.sec_model_df()
sec_model.head()

,ticker,periodOfReport,Business,Risk Factors,Management’s Discussion and Analysis of Financial Condition and Results of Operations,All Text
0,NFLX,2022-03-31,NaN,undefined,Item 2. Management&#8217;s Discussion and Ana...,undefined Item 2. Management&#8217;s Discussio...
1,NFLX,2021-09-30,NaN,undefined,Item 2. Management&#8217;s Discussion and Ana...,undefined Item 2. Management&#8217;s Discussio...
2,NFLX,2021-06-30,NaN,undefined,Item 2. Management&#8217;s Discussion and Ana...,undefined Item 2. Management&#8217;s Discussio...
3,NFLX,2021-03-31,NaN,undefined,Item 2. Management&#8217;s Discussion and Ana...,undefined Item 2. Management&#8217;s Discussio...
4,NFLX,2020-09-30,NaN,undefined,Item 2. \n\nManagement&#8217;s Discussion and...,undefined Item 2. \n\nManagement&#8217;s Discu...


In [72]:
thatLongColumnTitle = 'Management’s Discussion and Analysis of Financial Condition and Results of Operations'

In [73]:
# Code analyzing undefined sections. After removing addendums where ALL sections are undefined the remaining undefined are not hurtful
undefined_business = sec_text_df[sec_text_df['Risk Factors']=='undefined']
undefined = undefined_business[undefined_business[thatLongColumnTitle]=='undefined']
undefined['linkToFilingDetails'].iloc[0]

'https://www.sec.gov/Archives/edgar/data/867773/000086777313000020/spwr1230201210-ka.htm'

In [74]:
#removing special characters '\n' and '\t'
def remove_specials(df:pd.DataFrame):
    df = df.replace(r'\r+|\n+|\t+','', regex=True)
sec_text_df["Business"].replace(r'\r+|\n+|\t+','', regex=True,inplace=True)
sec_text_df["Risk Factors"].replace(r'\r+|\n+|\t+','', regex=True,inplace=True)
sec_text_df[thatLongColumnTitle].replace(r'\r+|\n+|\t+','', regex=True,inplace=True)
sec_text_df.head()

,ticker,periodOfReport,linkToFilingDetails,formType,cik,Business,Risk Factors,Management’s Discussion and Analysis of Financial Condition and Results of Operations
0,NFLX,2022-03-31,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,NaN,undefined,Item 2. Management&#8217;s Discussion and Ana...
1,NFLX,2021-09-30,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,NaN,undefined,Item 2. Management&#8217;s Discussion and Ana...
2,NFLX,2021-06-30,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,NaN,undefined,Item 2. Management&#8217;s Discussion and Ana...
3,NFLX,2021-03-31,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,NaN,undefined,Item 2. Management&#8217;s Discussion and Ana...
4,NFLX,2020-09-30,https://www.sec.gov/Archives/edgar/data/106528...,10-Q,1065280,NaN,undefined,Item 2. Management&#8217;s Discussion and Ana...


In [75]:
sec_text_df.to_csv('.\Data\Cleaned_Data\sec_all_data.csv')